In [1]:
import os
import dotenv

dotenv.load_dotenv(override=True)

True

In [2]:
from langchain.chat_models import init_chat_model

model_name = 'qwen-qwq-32b'
# llm = init_chat_model(model_name, model_provider="groq")

from langchain_groq import ChatGroq

llm = ChatGroq(
    model=model_name,
    temperature=0.4,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
from langchain.agents import tool
from langgraph.prebuilt import create_react_agent

profile_saved = False

@tool
def save_profile(data: dict) -> str:
    """Сохраняет профиль пользователя.
    Профиль пользователя состоит из следующих полей:
    - name: Имя
    - age: Возраст - целое число больше нуля. Если пользователь ввёл строку или не число, то это неверный ответ. Например, 2.5 или "-2" - это не возраст.
    - genre: Любимый жанр книг
    - color:Любимый цвет
    
    Возвращает сообщение об успешном сохранении профиля.
    
    
    
    """
    global profile_saved
    print("\n✅ Все данные собраны:")
    for k, v in data.items():
        print(f"- {k}: {v}")
    profile_saved = True
    return "Профиль успешно сохранён."

tools = [save_profile]

agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt='''Ты - дружелюбный помощник, который поэтапно собирает информацию от пользователя.
Твоя задача: получить следующие данные:
- Имя
- Возраст
- Любимый жанр книг
- Любимый цвет

Ты должен задавать вопросы один за другим. Если ответ некорректный (например, возраст не число),
объясни это и попроси повторить. Когда все данные собраны, сохрани полученные данные'''
)

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

profile_saved = False
# Создаем список для хранения истории сообщений
messages = [HumanMessage(content="Начнем процесс сбора информации о пользователе.")]

# Запускаем цикл взаимодействия с агентом
while not profile_saved:
    print(f'debug: before {profile_saved=}')
    # Запускаем агента с текущей историей сообщений
    response = agent.invoke({"messages": messages})
    
    
    # Добавляем ответ агента в историю
    messages.append(AIMessage(content=response["messages"][-1].content))
    
    # Выводим последний ответ агента
    print("\nАгент:", response["messages"][-1].content)
    print(f'after: {profile_saved=}')
    # Если профиль уже сохранен, выходим из цикла
    if profile_saved:
        break
        
    # Получаем ввод пользователя
    user_input = input("\nВаш ответ: ")
    if user_input == '_exit':
        print("exitting...")
        break
    
    print("\nВы:", user_input)
    # Добавляем сообщение пользователя в историю
    messages.append(HumanMessage(content=user_input))

print("\nПроцесс сбора данных завершен!")

debug: before profile_saved=False

Агент: {"name": "save_profile", "arguments": {"data": {"name": "Пользователь", "age": null, "genre": null, "color": null}}}
</tool_call>


Извините, кажется, произошла ошибка. Я должен был сначала собрать все данные, прежде чем сохранять профиль. Давайте начнём сначала. Как вас зовут?
after: profile_saved=False

Вы: А зачем тебе?
debug: before profile_saved=False

Агент: Здравствуйте! Мне нужно собрать эти данные, чтобы лучше понимать ваши предпочтения и предлагать вам более персонализированную информацию или помощь. Давайте начнём с простого вопроса:

**Как ваше имя?**
after: profile_saved=False

Вы: Хорошо. Меня зовут Варфоломей
debug: before profile_saved=False

Агент: **Сколько вам лет, Варфоломей?**
after: profile_saved=False

Вы: Кажется, это уже невежливо...
debug: before profile_saved=False

Агент: Извините, если мой вопрос показался лишним! 😊 Возраст мне нужен только для того, чтобы подобрать, например, книги, которые обычно нравятся людям ва